In [5]:
from IPython.display import display
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib as mpl
import warnings
warnings.filterwarnings("ignore")

plt.style.use("fivethirtyeight")

sns.set_style({'font.sans-serif': ['simsun', 'Arial']})
sns.set_style('darkgrid', {'font.sans-serif': ['simhei', 'Arial']})
%matplotlib inline

# 专利.xlsx

In [43]:
df_zhuanli = pd.read_excel(r'../../Data/CompInfo/专利.xlsx')

In [44]:
df_zhuanli.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 980360 entries, 0 to 980359
Data columns (total 4 columns):
企业编号     980360 non-null int64
专利类型     980360 non-null object
授权公告日    980360 non-null object
申请日      980360 non-null object
dtypes: int64(1), object(3)
memory usage: 29.9+ MB


In [45]:
df_zhuanli_copy = df_zhuanli.copy()

* 授权公告日是指专利部门作出授予专利权的决定，发给专利证书，同时予以登记和公告的日子。专利权自公告之日起生效。

In [46]:
display(df_zhuanli.head(n=5))

,企业编号,专利类型,授权公告日,申请日
0,1001,发明公布,2006-09-08­,2008-03-12
1,1001,发明公布,2006-06-23­,2007-12-26
2,1001,发明公布,2006-06-23,2007-12-26
3,1001,发明公布,2006-09-08,2008-03-12
4,1002,实用新型,2010-09-26,2011-10-26


## 专利类型
统计每个公司的每个专利数量

In [47]:
df_zhuanli['专利类型'].value_counts()

发明公布      404207
实用新型      300250
发明授权      170671
外观设计      104337
发明公布更正       652
发明专利         107
实用新型更正        68
外观设计更正        38
发明授权更正        30
Name: 专利类型, dtype: int64

In [48]:
df_zhuanli['专利类型'] = df_zhuanli['专利类型'].str.replace('更正', '', regex=True)

In [49]:
df_zhuanli['专利类型'].value_counts()

发明公布    404859
实用新型    300318
发明授权    170701
外观设计    104375
发明专利       107
Name: 专利类型, dtype: int64

In [54]:
df_zhuanli['授权公告日'] = df_zhuanli['授权公告日'].str.replace(
    r'[^0-9\-]', '', regex=True)
df_zhuanli['申请日'] = df_zhuanli['申请日'].str.replace(r'[^0-9\-]', '', regex=True)

df_zhuanli['授权公告日'] = pd.to_datetime(df_zhuanli['授权公告日'])
df_zhuanli['申请日'] = pd.to_datetime(df_zhuanli['申请日'])
df_zhuanli['专利申请时间差'] = df_zhuanli['申请日'].sub(
    df_zhuanli['授权公告日'], axis=0).dt.days
display(df_zhuanli.head(n=5))

,企业编号,专利类型,授权公告日,申请日,专利申请时间差
0,1001,发明公布,2006-09-08,2008-03-12,551
1,1001,发明公布,2006-06-23,2007-12-26,551
2,1001,发明公布,2006-06-23,2007-12-26,551
3,1001,发明公布,2006-09-08,2008-03-12,551
4,1002,实用新型,2010-09-26,2011-10-26,395


In [58]:
df_zhuanli = df_zhuanli.drop(columns=['授权公告日', '申请日'])
display(df_zhuanli.head(n=5))

,企业编号,专利类型,专利申请时间差
0,1001,发明公布,551
1,1001,发明公布,551
2,1001,发明公布,551
3,1001,发明公布,551
4,1002,实用新型,395


In [ ]:
# new_df_zhuanli = df_zhuanli.groupby(['企业编号', '专利类型'])['专利类型'].count().unstack()
# new_df_zhuanli.columns = ['发明专利', '发明公布', '发明授权', '外观设计', '实用新型']
# new_df_zhuanli['企业编号'] = new_df_zhuanli.index
# columns = ['企业编号', '发明专利', '发明公布', '发明授权', '外观设计', '实用新型']
# new_df_zhuanli = pd.DataFrame(new_df_zhuanli, columns=columns)
# new_df_zhuanli = new_df_zhuanli.set_index(np.arange(0, len(new_df_zhuanli)))
# new_df_zhuanli.head(n=5)